# Load data

In [1]:
from pymatgen import Structure
from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]

# Set up the GAP and train

In [2]:
from mlearn.potentials.gap import GAPotential

gap = GAPotential()
gap.train(train_structures=train_structures, energies=train_energies,
          forces=train_forces, sparse_method='cur_points', 
          l_max=8, n_max=8, e0=-0.97, n_sparse=200, cutoff=5.0)

0

# Lattice constant, Elastic constant

In [3]:
from mlearn.potentials.lammps.calcs import LatticeConstant

conventional_cell = Structure.from_file('conventional.cif')
lc_calculator = LatticeConstant(ff_settings=gap)
a, b, c = lc_calculator.calculate([conventional_cell])[0]
print('Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

Lattice a: 3.09752663097197, Lattice b: 3.09752663097197, Lattice c: 3.09752663097197


In [4]:
from mlearn.potentials.lammps.calcs import ElasticConstant

ec_calculator = ElasticConstant(ff_settings=gap, lattice='bcc', alat=3.106)
C11, C12, C44, bulk_modulus = ec_calculator.calculate()
print('C11: {}, C12: {}, C44: {}, bulk modulus: {}'.format(C11, C12, C44, bulk_modulus))

C11: 825.973623413894, C12: 520.048832606182, C44: 108.887418997229, bulk modulus: 622.023761105942


# Load model from *.xml

In [5]:
from pymatgen import Specie

gap_loaded = GAPotential.from_config('soap.xml')
gap_loaded.specie = Specie('Mo')

# Energy, force, stress prediction

In [6]:
from mlearn.potentials.lammps.calcs import EnergyForceStress

struct = Structure.from_file('test_struct.cif')
efs_calculator = EnergyForceStress(ff_settings=gap_loaded)
energy, forces, stresses = efs_calculator.calculate([struct])[0]

print('energy: {}'.format(energy))
print('forces: \n', forces)
print('stresses: ', stresses)

energy: -470.226483634706
forces: 
 [[ 5.17220e-01 -2.26974e+00  3.27873e+00]
 [-6.49528e-01  4.57412e-03 -1.41823e-01]
 [ 1.51153e+00 -2.94829e+00  3.34419e+00]
 [ 2.12736e+00  9.95482e-01 -1.37877e+00]
 [-2.49626e-01  9.50299e-01 -1.03167e+00]
 [ 1.33313e+00 -2.25837e+00 -9.39224e-01]
 [ 1.85123e+00 -4.17418e+00 -3.30936e+00]
 [ 4.79854e-01 -1.85012e-01 -8.00007e-01]
 [-1.58940e+00  2.14939e+00 -2.11070e-01]
 [-4.45454e+00  4.20825e+00  3.84693e+00]
 [ 2.78797e+00 -9.69087e-01  2.26335e+00]
 [ 2.39265e+00  1.21409e+00 -2.09260e+00]
 [ 3.37828e-01 -6.94593e-01 -1.39380e+00]
 [-2.13208e+00  1.14523e-01 -7.58938e-02]
 [-3.20492e+00  3.38092e+00 -6.10804e-01]
 [-1.95323e+00  5.38365e-01 -6.36190e-01]
 [ 1.34853e+00  1.13590e+00  2.50862e+00]
 [ 2.05813e+00 -3.54593e-01  1.75672e+00]
 [ 5.43265e-01  1.26891e+00  7.31121e-01]
 [-3.00155e+00  6.34753e-01  2.64025e+00]
 [ 5.96724e-02 -4.38550e+00  7.04372e-01]
 [-3.27469e+00 -1.46320e+00 -6.48063e-01]
 [ 5.61371e-01 -2.60958e-02 -1.63941e+00